Fake news is very serious issue in today's society.Many social sites not fact checking the news it's very easy for anyone to spread rumours ,hoaxes  or something derogatory remarks again anybody without any proper proof.When identifying a source of information, one must look at many attributes, including but not limited to the content of the email and social media engagements. specifically, the language is typically more inflammatory in fake news than real.


## The data
There are two csv files namely "true.csv" and "False.csv".First we have to merge both of them into a single dataset.Then we will shuffle it .The columns in both data are same namely:
* title   : the title of article
* text    : the text of article
* subject : the subject of article
* date    : the date on which article posted

One more column that i added is target which contain the class of news."1" for fake news and "0" for true news.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Imports 

In [ ]:
from sklearn.utils import shuffle
import re
import string
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import plot_confusion_matrix,precision_score,recall_score
from sklearn.model_selection import cross_val_score,train_test_split


## Reading Data 

In [ ]:
fake=pd.read_csv("/kaggle/input/fake-and-real-news-dataset/Fake.csv")
true=pd.read_csv("/kaggle/input/fake-and-real-news-dataset/True.csv")

In [ ]:
#making a news target column
fake['target']=1
true['target']=0

In [ ]:
Combine=pd.concat([fake,true])
#shuffling the data 
df=shuffle(Combine)
df=df.reset_index()

In [ ]:
print(f"shape of dataframe:{df.shape}")

**Checking for Nans**

In [ ]:
df.isna().sum()

In [ ]:
df.info()

Take a look at the top 10 rows of the merged dataset

In [ ]:
df.head(10)

## Preprocessing  

In this part we will preprocess the data to make it more efficient to consumed by our model, We will standardised the text into lowercase ,remove all the text inbetween  square brackets, remove all the words tha contain digits or punctuation in between.When we remove the 
inverted comma from word "don't" then we are left with "don t" so i should remove the " t " from this also.

In [ ]:
def clean_text(text):
    text=text.lower()
    text=re.sub(f"[{string.punctuation}]"," ",text)
    text=re.sub("\[.*?\]"," ",text)   
    text=re.sub("\w\d\w"," ",text)
    text=re.sub("\(.*?\)"," ",text)
    text=re.sub("[\"\"]"," ",text) 
    text=re.sub(' t ',' ',text)
    text=re.sub(' s ',' ',text)
    text=re.sub("[^a-zA-Z]"," ",text)
    return text

In [ ]:
#applying the process
df["text"]=df['text'].apply(lambda x:clean_text(x))
df['title']=df['title'].apply(lambda x:clean_text(x))

Take a look at the some random rows of "title" after cleaning

In [ ]:
df['title'][443:579]

In [ ]:
X=df['title']
Y=df['target']


 Reserving a small part of data to check the performance  of my model

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.1)

In [ ]:
Count_vec=CountVectorizer(stop_words='english',max_features=60)

Since i didn't remove the stopwords at the preprocessing step so when using countvectorizer i mention the stopwords parameter in countvectorizer which when creating features didn't consider stopwords. 

In [ ]:
X_Count=Count_vec.fit_transform(x_train).toarray()

In [ ]:
Count_vec.get_feature_names()[:12]

In [ ]:
X_Count.shape

In [ ]:
Count_df=pd.DataFrame(X_Count,columns=Count_vec.get_feature_names())

In [ ]:
Count_df.head(10)

## Model Building

In [ ]:
naive_bayes_Countvec=MultinomialNB()
scores_cout=cross_val_score(naive_bayes_Countvec,X_Count,y_train,cv=5,scoring='f1')

In [ ]:
scores_cout

So we get a decent  score using Naive bayes on Countvectorizer.Now let's fit the model   

In [ ]:
naive_bayes_Countvec.fit(X_Count,y_train)

So after using countvectorizer let's try tf-idfvectorizer to vectorize the text and compare the results with Naive bayes. 

In [ ]:
naive_bayes_tfidf=MultinomialNB()
tfidf_vec=TfidfVectorizer(stop_words='english')
X_tf=tfidf_vec.fit_transform(x_train)
scores_tf=cross_val_score(naive_bayes_tfidf,X_tf,y_train,cv=5,scoring='f1')

In [ ]:
scores_tf

So with tf-idf vectorizer i get much better results.Let's fit the model.

In [ ]:
naive_bayes_tfidf.fit(X_tf,y_train)

In [ ]:
test_count=Count_vec.transform(x_test)
plot_confusion_matrix(naive_bayes_Countvec,test_count,y_test)


Above the numbers at diagonal shows the correctly classified data.So our model correctly  classified  1243 true news as true and 2131 false as false and 293 false news as true and 823 true news as false.This is when i use Countvectorizer to vectorize the text.    

In [ ]:
y_pred_count=naive_bayes_Countvec.predict(test_count)

In [ ]:
print("Naive_Bayes_Model with Countvectorizer : \n")
print(f"precision:{precision_score(y_test,y_pred_count)} \n recall:{recall_score(y_test,y_pred_count)}")

In [ ]:
test_tf=tfidf_vec.transform(x_test)
plot_confusion_matrix(naive_bayes_tfidf,test_tf,y_test)

So our precision  and recall  gets improved when using tf-idfvectorizer.In other words the tendency of model to correctly classsify the class of news gets improved.

In [ ]:
y_pred_tf=naive_bayes_tfidf.predict(test_tf)


In [ ]:
print("Naive_Bayes_model with Tfidf: \n")
print(f"precsion:{precision_score(y_test,y_pred_tf)} \n recall:{recall_score(y_test,y_pred_tf)}")

If you like this kernel please show your support by upvoting this kernel. As your upvotes encourage me do more on Kaggle.If you find any mistake or any suggestion to make this kernel better  let me know in comments below.   